## Predicting Protein Function: A Comparison of CNN and Transformer Models

* **Author:** Hosein Mohammadi
* **Date:** July 2024
* **Contact:** [huseinmohammadi83@gmail.com](mailto:huseinmohammadi83@gmail.com)
* **LinkedIn:** [Hosein Mohammadi](https://www.linkedin.com/in/hosein-mohammadi-979b8a2b2/)
* **Project Repository:** [protein-function-prediction](https://github.com/Hosein541/protein-function-prediction)

---

### Project Overview

This notebook contains the complete code for the "Protein Function Prediction" project. The primary goal is to predict a protein's functions (represented by Gene Ontology terms) directly from its amino acid sequence.

To achieve this, we implement and rigorously compare two distinct deep learning architectures:

1.  A **1D Convolutional Neural Network (CNN)** built from scratch in Keras to serve as a strong baseline.
2.  A fine-tuned **Transformer model (ESM-2)** from Hugging Face, leveraging the power of transfer learning from a model pre-trained on millions of protein sequences.

The notebook covers all steps from data acquisition and preprocessing (using data from UniProt) to model training, hyperparameter optimization (threshold tuning), and a final comparative analysis of the results.1mm

In [ ]:
# =======================================================
#                      IMPORTS
# =======================================================

# --- Standard Libraries ---
import os
import re
from collections import Counter

# --- Data Manipulation & Plotting ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- Scikit-Learn (General ML & Metrics) ---
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, roc_auc_score, classification_report

# --- TensorFlow / Keras (for the CNN model) ---
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.preprocessing.sequence import pad_sequences

# --- PyTorch & Hugging Face (for the Transformer model) ---
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

# --- Environment Setup ---
# Disable W&B logging for a cleaner run
os.environ["WANDB_DISABLED"] = "true"

print("All libraries imported successfully!")

### 1. Data Loading and Preprocessing

In this first step, we load the dataset sourced from UniProt and perform the initial cleaning required to prepare it for our models.

The process includes:
* **Loading Data:** The `.tsv.gz` file is loaded into a pandas DataFrame.
* **Initial Inspection:** We use `.head()` and `.info()` to get a first look at the data structure and identify any missing values.
* **Cleaning:**
    * We remove any rows that are missing a protein sequence or Gene Ontology (GO) term, as they are unusable for training.
    * We remove any proteins with duplicate sequences to prevent data redundancy.

In [ ]:
# 1. Place the name of the file you downloaded from UniProt here
file_path = "uniprotkb_reviewed_true_AND_organism_id_2025_07_29.tsv.gz"

# 2. Load data
# Since our file is tab-separated, we use sep='\t'
try:
    df = pd.read_csv(file_path, sep='\t')
except FileNotFoundError:
    print(f"Error: File '{file_path}' not found. Please check the file name and path.")
    exit()


# 3. Display the first 5 rows for initial inspection
print("--- First 5 rows of data ---")
print(df.head())
print("\n" + "="*30 + "\n")

# 4. Display general DataFrame information (important)
# This section tells us how many rows of data we have and whether there are any empty (NaN) values
print("--- General DataFrame information (before cleaning) ---")
df.info()
print("\n" + "="*30 + "\n")


# 5. Initial cleaning
# Remove rows that have empty values in the 'Sequence' or 'Gene Ontology (GO)' columns
df.dropna(subset=['Sequence', 'Gene Ontology (GO)'], inplace=True)

# Remove duplicate rows based on protein sequence
# Sometimes different proteins have identical sequences, which are considered duplicates for our model
initial_rows = len(df)
df.drop_duplicates(subset=['Sequence'], inplace=True)
print(f"{initial_rows - len(df)} duplicate rows removed.")
print("\n" + "="*30 + "\n")


# 6. Display final information after cleaning
print("--- General DataFrame information (after cleaning) ---")
print("Final DataFrame shape:", df.shape)
df.info()

### 2. Label Processing: From Text to Binary Vectors

This section transforms the raw `Gene Ontology (GO)` text column into a machine-learning-ready format. The goal is to create a binary matrix where each row represents a protein and each column represents a specific function.

This process involves several key steps:

1.  **Extract GO Codes:** We use **regular expressions** to parse the text and extract all valid GO codes (e.g., `GO:0005737`).
2.  **Identify Top Labels:** To make the classification problem manageable, we count all the extracted GO terms and select the **1,000 most common** ones to act as our target labels. The number of labels is a **hyperparameter** that can be tuned.
3.  **Filter the Dataset:** We remove any proteins that do not have at least one of these top 1,000 GO terms.
4.  **Binarize Labels:** We use Scikit-Learn's **`MultiLabelBinarizer`** to convert the list of GO terms for each protein into a binary vector (a process also known as one-hot encoding). A `1` in a column indicates the protein has that function, and a `0` indicates it does not.

In [ ]:
# 1. Extract GO codes from text string
def extract_go_terms(text):
    # This regex finds all strings that start with "GO:" and have 7 digits
    return re.findall(r'GO:\d{7}', str(text))

print("Extracting GO codes...")
df['go_terms'] = df['Gene Ontology (GO)'].apply(extract_go_terms)

# Remove rows where no valid GO code was found
df = df[df['go_terms'].apply(len) > 0]
print(f"Number of remaining proteins after removing rows without GO: {len(df)}")
print("\n" + "="*30 + "\n")


# 2. Find the most common GO terms
# Collect all lists of GO codes into one large list
all_go_terms = [term for terms_list in df['go_terms'] for term in terms_list]

# Count the occurrences of each GO code
go_counts = Counter(all_go_terms)

# Determine the number of most common labels (e.g., the first 1000)
# This number is a hyperparameter and can be changed
NUM_LABELS = 1000
top_go_terms = [term for term, count in go_counts.most_common(NUM_LABELS)]
print(f"Total number of unique GO codes: {len(go_counts)}")
print(f"{NUM_LABELS} most common GO codes selected for modeling.")
print("Example of common codes:", top_go_terms[:10])
print("\n" + "="*30 + "\n")


# 3. Filter DataFrame
# Keep only proteins that have at least one of the most common labels
def filter_top_go(terms):
    return [term for term in terms if term in top_go_terms]

df['go_terms_filtered'] = df['go_terms'].apply(filter_top_go)
df = df[df['go_terms_filtered'].apply(len) > 0]
print(f"Number of final proteins for modeling: {len(df)}")
print("\n" + "="*30 + "\n")


# 4. Convert to binary format (Multi-Label Binarization)
print("Converting labels to binary format...")
mlb = MultiLabelBinarizer(classes=top_go_terms)
y = mlb.fit_transform(df['go_terms_filtered'])

# Convert the binary matrix to a DataFrame for better viewing
labels_df = pd.DataFrame(y, columns=mlb.classes_)

print("Shape of the labels matrix (number of samples, number of labels):", labels_df.shape)
print("Example of the final labels matrix:")
print(labels_df.head())

# Now 'df' contains sequences and 'labels_df' contains labels ready for the model.
# You can save these two for subsequent steps.
# df.to_csv('processed_sequences.csv', index=False)
# labels_df.to_csv('processed_labels.csv', index=False)

### 3. Sequence Preparation: Tokenization and Padding

In this step, we convert the protein sequences from text strings into fixed-size numerical matrices that can be fed into a deep learning model. This is a standard procedure in sequence-based modeling.

The process involves two main parts:

1.  **Tokenization:** First, we create a **vocabulary** that maps each unique amino acid character to an integer. We then use this vocabulary to convert each protein sequence into a list of numbers.

2.  **Padding:** Deep learning models require their inputs to be of a uniform size. Since protein sequences have varying lengths, we standardize them. We calculate the **95th percentile** of all sequence lengths and use this as our maximum length (`MAX_LEN`). Sequences longer than this are truncated, and shorter sequences are "padded" with a special token (0) until they reach this length.

In [ ]:
# 1. Build dictionary for amino acids
# List of all standard amino acids + a token for unknown amino acids ('U')
# and a token for padding ('X') which takes the value zero
amino_acids = 'ACDEFGHIKLMNPQRSTVWYU'
vocab = {aa: i + 1 for i, aa in enumerate(amino_acids)}
vocab['X'] = 0  # Padding token

VOCAB_SIZE = len(vocab)
print(f"Vocabulary Size: {VOCAB_SIZE}")
print("Amino acid dictionary:", vocab)
print("\n" + "="*30 + "\n")


# 2. Convert sequences to numerical lists (tokenization)
def tokenize_sequence(sequence):
    return [vocab.get(aa, vocab['U']) for aa in sequence] # If amino acid is not found, consider it 'U'

print("Tokenizing sequences...")
df['sequence_tokenized'] = df['Sequence'].apply(tokenize_sequence)
print("Example of a tokenized sequence:")
print(df['sequence_tokenized'].iloc[0])
print(df['Sequence'].iloc[0])
print("\n" + "="*30 + "\n")


# 3. Standardize sequence length (Padding)
# Find a suitable length for all sequences (e.g., 95th percentile)
# This prevents excessive length due to a few very long sequences
MAX_LEN = int(df['Sequence'].str.len().quantile(0.95))
print(f"Maximum length for padding (95th percentile): {MAX_LEN}")

# Apply padding
# pre: padding is added to the beginning of the sequence
X = pad_sequences(df['sequence_tokenized'], maxlen=MAX_LEN, padding='post', truncating='post')

print("Final input matrix shape (number of samples, sequence length):", X.shape)
print("Example of a sequence after padding:")
print(X[0])

# Now matrix 'X' contains inputs ready for the model and matrix 'y' (from previous step) are our outputs.

### 4. Baseline Model: 1D-CNN

With the data fully prepared, we now define, compile, and train our first model. This **1D-Convolutional Neural Network (CNN)** will serve as our **baseline**, providing a performance benchmark that our more advanced Transformer model will need to surpass.

The process involves these steps:

* **Data Splitting:** We first divide our data into training (80%) and testing (20%) sets to ensure we can evaluate the model's performance on unseen data.

* **Model Architecture:** We build the model using the Keras `Sequential` API. The key layers are:
    * An **`Embedding`** layer, which learns a dense vector representation for each amino acid.
    * A **`Conv1D`** layer, which is highly effective at scanning for and learning local patterns (motifs) within the protein sequences.
    * A final **`Dense`** output layer with a **`sigmoid`** activation function, which is essential for multi-label classification as it allows the model to predict a probability for each of the 1,000 labels independently.

* **Compilation:** The model is compiled using the `adam` optimizer and the **`binary_crossentropy`** loss function, which is the standard choice for multi-label classification problems.

* **Training:** Finally, the model is trained on the training data for 10 epochs.

In [ ]:
# --- Assuming matrices X and y are ready from previous steps ---
# X: Tokenized and padded sequence matrix
# y: Binary label matrix

# 1. Split data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of training data:", X_train.shape, y_train.shape)
print("Shape of test data:", X_test.shape, y_test.shape)
print("\n" + "="*30 + "\n")


# 2. Define model parameters
# These values were calculated in previous steps
# VOCAB_SIZE = len(vocab)
# MAX_LEN = int(df['Sequence'].str.len().quantile(0.95))
# NUM_LABELS = 1000
EMBEDDING_DIM = 128  # Dimension of the vector learned for each amino acid


# 3. Build model architecture
print("Building 1D-CNN model...")
model = Sequential([
    # Embedding layer: Converts each number (token) into a dense vector
    Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_LEN),

    # Convolutional layer to find patterns in the sequence
    Conv1D(filters=128, kernel_size=5, activation='relu'),

    # Pooling layer to reduce dimensionality and extract most important features
    GlobalMaxPooling1D(),

    # Fully connected layer for learning feature combinations
    Dense(256, activation='relu'),
    Dropout(0.5), # To prevent overfitting

    # Output layer: Number of neurons equals the number of labels
    # Sigmoid activation is essential for multi-label classification
    Dense(NUM_LABELS, activation='sigmoid')
])

# 4. Compile the model
# binary_crossentropy loss function is suitable for multi-label classification
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[
                  AUC(name='auc'),
                  Precision(name='precision'),
                  Recall(name='recall')
              ])


# 5. Train the model
print("Starting model training process...")
EPOCHS = 10
BATCH_SIZE = 64

history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_test, y_test)
)

print("\nBase model training completed successfully!")

### 5. Baseline Model Evaluation

After training, we evaluate the performance of our baseline CNN on the unseen **test set**. This step is crucial to understand how well the model generalizes to new data.

The evaluation process includes:
* **Generating Predictions:** The trained model predicts a probability score (between 0 and 1) for each of the 1,000 possible functions for every protein in the test set.
* **Applying a Threshold:** We use a decision **threshold** to convert these probabilities into binary predictions (1 for "has the function," 0 for "does not").
* **Calculating Metrics:** We calculate two primary metrics to assess performance:
    * **AUC (Area Under the Curve):** A threshold-independent metric that measures the model's overall ability to correctly rank positive labels higher than negative ones.
    * **F1-Score:** A threshold-dependent metric that provides a balanced measure of a model's precision and recall.

In [ ]:
# 1. Get model predictions on test data
y_pred_probs = model.predict(X_test)

# 2. Convert probabilities to binary labels (0 and 1)
THRESHOLD = 0.3
y_pred = (y_pred_probs > THRESHOLD).astype(int)

# 3. Calculate final metrics using 'micro' average to prevent errors
print(f"Prediction Threshold: {THRESHOLD}\n")

# 'micro' averaging is robust to labels not present in the test set
f1_micro = f1_score(y_test, y_pred, average='micro', zero_division=0)
auc_micro = roc_auc_score(y_test, y_pred_probs, average='micro')

print(f"Final AUC Score (Micro): {auc_micro:.4f}")
print(f"Final F1-Score (Micro): {f1_micro:.4f}")

### 6. Advanced Model: Fine-Tuning a Transformer

Now we move to our second approach: using a large, pre-trained **Transformer** model. Instead of building a model from scratch, we will use **transfer learning**. We will take a model that has already learned the "language" of proteins from millions of sequences and **fine-tune** it for our specific prediction task.

This section covers loading the model and its tokenizer from the **Hugging Face** Hub.

* **Model Selection:** We use **ESM-2** (`facebook/esm2_t6_8M_UR50D`), a powerful protein language model developed by Meta AI.
* **Tokenizer and Model Loading:** We load the pre-trained model weights and the specific tokenizer that was trained with it.
* **Model Customization:** We adapt the pre-trained model for our task by replacing its original classification layer with a new one designed for our 1,000-label, multi-label classification problem.

In [ ]:
# 1. Define the name of the model we want to use from Hugging Face
# We are using a small version of ESM-2 called t6_8M, which is suitable for starting
model_name = "facebook/esm2_t6_8M_UR50D"

# 2. Load the specific tokenizer for this model
# This tokenizer knows exactly how to prepare the protein sequence for ESM-2
tokenizer = AutoTokenizer.from_pretrained(model_name)
NUM_LABELS = 1000
# 3. Load the pre-trained model
# We tell the model that we have a multi-label classification problem
transformer_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=NUM_LABELS,  # This variable is our number of labels from the previous section (e.g., 1000)
    problem_type="multi_label_classification"
)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transformer_model.to(device)

print(f"Model '{model_name}' successfully loaded and moved to device '{device}'.")

### 7. Data Preparation for the Transformer

The Hugging Face Transformer model requires data to be in a specific format for training. This step uses the model's own **tokenizer** and a custom **PyTorch Dataset** class to prepare the data.

* **Tokenization:** We re-tokenize all protein sequences using the specific **ESM-2 tokenizer**. This process converts the sequences into numerical `input_ids` and also creates an `attention_mask`, which tells the model which tokens are real amino acids versus padding.
* **PyTorch Dataset:** A custom `ProteinGODataset` class is created to package our tokenized inputs and binary labels together. This is the standard format required by the Hugging Face `Trainer` API for handling data during training and evaluation.

In [ ]:
# --- Assuming 'df' and 'labels_df' DataFrames are ready from Section 2 ---
# Reset indices to ensure consistency
df.reset_index(drop=True, inplace=True)
labels_df.reset_index(drop=True, inplace=True)

# 1. Tokenize all sequences with the ESM-2 tokenizer
print("Tokenizing sequences with Hugging Face tokenizer...")
tokenized_inputs = tokenizer( # Assuming 'tokenizer' is defined elsewhere (e.g., loaded ESM-2 tokenizer)
    df['Sequence'].tolist(),
    max_length=MAX_LEN,   # Using the same MAX_LEN as before (assuming it's defined)
    padding='max_length', # Pad to maximum length
    truncation=True,      # Truncate longer sequences
    return_tensors='pt'   # Return PyTorch tensors
)

# 2. Build a custom PyTorch Dataset class
# This class prepares the data in the format required by Hugging Face Trainer
class ProteinGODataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Returns a dictionary of required tensors for a single data sample
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        # Labels must be of float type
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

# 3. Build training and test datasets
# For a fair comparison, use the same random_state as before for data splitting
indices = range(len(df))
train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)

# Convert labels DataFrame to a NumPy array
y_numpy = labels_df.values

# Build training dataset
train_dataset = ProteinGODataset(
    {key: val[train_indices] for key, val in tokenized_inputs.items()},
    y_numpy[train_indices]
)

# Build test dataset
test_dataset = ProteinGODataset(
    {key: val[test_indices] for key, val in tokenized_inputs.items()},
    y_numpy[test_indices]
)


print("\nData successfully converted to PyTorch Dataset format.")
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")

### 8. Transformer Fine-Tuning and Evaluation

This is the final training step where we **fine-tune** the pre-trained ESM-2 model on our specific dataset. We use the high-level **`Trainer`** API from Hugging Face, which automates the entire process.

* **Training Arguments:** We first define all the **hyperparameters** for our training run, such as the number of epochs, learning rate, and batch size, using the `TrainingArguments` class.
* **Metrics Function:** A custom function, `compute_metrics`, is created to calculate the F1-Score and AUC on the validation set at the end of each epoch, allowing us to monitor the model's performance.
* **Training:** The `Trainer` object brings together the model, data, and training arguments. The `trainer.train()` command launches the fine-tuning process.
* **Final Evaluation:** After training is complete, we evaluate the best-performing version of the model on our unseen test set to get the final performance scores.

In [ ]:
# 1. Define a function to compute metrics during evaluation
def compute_metrics(p):
    # p is a tuple of predictions and labels
    preds = p.predictions
    labels = p.label_ids

    # Apply sigmoid to get probabilities and find a threshold
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(preds))

    # We use the same 0.3 threshold as our baseline for a fair comparison
    y_pred = (probs > 0.3).numpy().astype(int)
    y_true = labels.astype(int)

    # Calculate micro-averaged F1 and AUC scores
    f1_micro = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    auc_micro = roc_auc_score(y_true=y_true, y_score=probs, average='micro')

    # Return metrics as a dictionary
    return {
        'f1_micro': f1_micro,
        'auc_micro': auc_micro
    }


# 2. Define Training Arguments (Corrected Version)
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,

    # --- RENAMED ARGUMENTS FOR OLDER VERSIONS ---
    eval_strategy='epoch',     # Changed from evaluation_strategy
    save_strategy='epoch',           # Changed from save_strategy
    # ------------------------------------------

    load_best_model_at_end=True,
    metric_for_best_model='f1_micro',
    fp16=True,
)


# --- The rest of your code stays exactly the same ---
# 3. Instantiate the Trainer
trainer = Trainer(
    model=transformer_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# 4. Start the fine-tuning process!
print("Starting the fine-tuning process...")
trainer.train()
print("Fine-tuning complete!")

# 5. Evaluate the final best model
final_evaluation = trainer.evaluate()
print("\n--- Final Evaluation of the Best Model ---")
print(final_evaluation)

### 9. Final Analysis and Threshold Optimization
After the models are trained, the final step is to perform a detailed analysis of their performance on the unseen test set. This involves generating predictions and then finding the optimal decision threshold for each model to maximize its F1-Score.

This first code block handles generating the predictions from the fine-tuned Transformer model.

* **Inference**: We use the `trainer.predict()` method to run the model on our test_dataset. This outputs raw, unnormalized scores known as logits.

* **Probability Conversion**: A `sigmoid` function is applied to the logits to convert them into probabilities, which are values between 0 and 1 that can be used for evaluation.

In [ ]:
# 1. Get the raw predictions from the fine-tuned trainer on the test_dataset
raw_predictions = trainer.predict(test_dataset)

# The output contains the raw model outputs (logits)
# We need to convert them to probabilities using a sigmoid function
import torch

y_pred_logits = raw_predictions.predictions
sigmoid = torch.nn.Sigmoid()
y_pred_probs_transformer = sigmoid(torch.Tensor(y_pred_logits)).numpy()

print("Successfully generated probability predictions (y_pred_probs) from the Transformer model.")
print("Shape of the predictions:", y_pred_probs.shape)

### Finding the Optimal Decision Threshold

The F1-Score is highly sensitive to the decision threshold used to convert probabilities into binary predictions. To find the best possible performance for each of our models, the following cells iterate through a range of thresholds and plot the results.

This process is performed for both the **CNN baseline** and the **ESM-2 Transformer** to identify the **optimal threshold** that maximizes the F1-Score for each, ensuring a fair and complete comparison.

In [ ]:
# --- Assuming y_test and y_pred_probs_transformer are ready from previous steps ---
# y_pred_probs_transformer is the probability output from the transformer model

# Test different thresholds from 0.1 to 0.5
thresholds = np.arange(0.1, 0.5, 0.01)
f1_scores = []

for thresh in thresholds:
    # Convert probabilities to binary predictions with the new threshold
    y_pred_binary = (y_pred_probs_transformer > thresh).astype(int)
    # Calculate F1-Score
    f1 = f1_score(y_test, y_pred_binary, average='micro', zero_division=0)
    f1_scores.append(f1)

# Find the best threshold
best_threshold_index = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_index]
best_f1_score = f1_scores[best_threshold_index]

print(f"Best threshold found: {best_threshold:.2f}")
print(f"Best possible F1-Score: {best_f1_score:.4f}")

# Plot the graph for better analysis
plt.figure(figsize=(10, 6))
plt.plot(thresholds, f1_scores, marker='o')
plt.title('ESM-2 Model : F1-Score vs. Threshold')
plt.xlabel('Threshold')
plt.ylabel('F1-Score (Micro)')
plt.grid(True)
plt.axvline(x=best_threshold, color='r', linestyle='--', label=f'Best Threshold = {best_threshold:.2f}')
plt.legend()
plt.show()

In [ ]:
# --- Assuming y_test and y_pred_probs are ready from previous steps ---
# y_pred_probs is the probability output from the Baseline model

# Test different thresholds from 0.1 to 0.5
thresholds = np.arange(0.1, 0.5, 0.01)
f1_scores = []

for thresh in thresholds:
    # Convert probabilities to binary predictions with the new threshold
    y_pred_binary = (y_pred_probs_transformer > thresh).astype(int)
    # Calculate F1-Score
    f1 = f1_score(y_test, y_pred_binary, average='micro', zero_division=0)
    f1_scores.append(f1)

# Find the best threshold
best_threshold_index = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_index]
best_f1_score = f1_scores[best_threshold_index]

print(f"Best threshold found: {best_threshold:.2f}")
print(f"Best possible F1-Score: {best_f1_score:.4f}")

# Plot the graph for better analysis
plt.figure(figsize=(10, 6))
plt.plot(thresholds, f1_scores, marker='o')
plt.title('CNN Model : F1-Score vs. Threshold')
plt.xlabel('Threshold')
plt.ylabel('F1-Score (Micro)')
plt.grid(True)
plt.axvline(x=best_threshold, color='r', linestyle='--', label=f'Best Threshold = {best_threshold:.2f}')
plt.legend()
plt.show()

### 10. Conclusion and Final Thoughts

This project successfully demonstrated an end-to-end workflow for predicting protein function from amino acid sequences using deep learning. We developed, trained, and rigorously compared two distinct models: a baseline 1D-CNN and a fine-tuned ESM-2 Transformer.

After optimizing the decision threshold for both models, our final results showed that the **1D-CNN baseline model slightly outperformed the pre-trained ESM-2 Transformer** on both primary metrics.

The key takeaway is a crucial lesson in applied machine learning: a well-designed, simpler architecture can be more effective than a large, complex pre-trained model, especially on specific, moderately-sized datasets. This result underscores the importance of **always establishing a strong baseline** and highlights that the most advanced model is not always the best solution for every problem.